In [1]:
²import tensorflow as tf
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
dataDirectory = "imgs/" # folder that contains the dataset

In [3]:
img_size = 224 # mobilenetv2 input size
classes = ["0", "1", "2", "3", "4", "5", "6"] #list of classes
trainingData = []

In [4]:
def create_training_data():
    for category in classes:
        img_path = os.path.join(dataDirectory, category)
        for img in os.listdir(img_path):
            try:
                img_array = cv2.imread(os.path.join(img_path, img))
                resized_img = cv2.resize(img_array, (img_size, img_size))
                trainingData.append([resized_img, int(category)])
            except Exception as e :
                pass

In [5]:
create_training_data()

In [6]:
import random
random.shuffle(trainingData)

In [7]:
X, y = [], []

for img, label in trainingData:
    X.append(img)
    y.append(label)

In [8]:
X = np.array(X).reshape(-1, img_size, img_size, 3)
X = X / 255.0 #normalize the data

y = np.array(y)

In [9]:
from tensorflow import keras
from tensorflow.keras import layers

In [10]:
model = tf.keras.applications.MobileNetV2()

In [11]:
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [12]:
input = model.layers[0].input
output = model.layers[-2].output

In [13]:
finalOutput = layers.Dense(128)(output) # adding dense layer after GlobalAveragePooling layer
finalOutput_ = layers.Activation('relu')(finalOutput)
finalOutput = layers.Dense(64)(finalOutput_)
finalOutput_ = layers.Activation('relu')(finalOutput)
finalOutput = layers.Dense(7, activation='softmax')(finalOutput_) # we are working with 7 classes

In [14]:
model = keras.Model(inputs= input, outputs= finalOutput)

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [16]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
model.fit(X, y, epochs=20)

Epoch 1/20
417/417 [==============================] - 467s 1s/step - loss: 1.4075 - accuracy: 0.4626
Epoch 2/20
417/417 [==============================] - 599s 1s/step - loss: 1.2083 - accuracy: 0.5480
Epoch 3/20
417/417 [==============================] - 644s 2s/step - loss: 1.1042 - accuracy: 0.5848
Epoch 4/20
417/417 [==============================] - 628s 2s/step - loss: 1.0386 - accuracy: 0.6127
Epoch 5/20
417/417 [==============================] - 598s 1s/step - loss: 0.9729 - accuracy: 0.6384
Epoch 6/20
417/417 [==============================] - 611s 1s/step - loss: 0.8959 - accuracy: 0.6690
Epoch 7/20
417/417 [==============================] - 518s 1s/step - loss: 0.8445 - accuracy: 0.6886
Epoch 8/20
417/417 [==============================] - 448s 1s/step - loss: 0.7749 - accuracy: 0.7175
Epoch 9/20
417/417 [==============================] - 460s 1s/step - loss: 0.7097 - accuracy: 0.7415
Epoch 10/20
417/417 [==============================] - 482s 1s/step - loss: 0.6511 - accura

In [18]:
model.save("final_model.h5")